In [2]:
import os
from pathlib import Path

import numpy as np
import xarray as xr

In [3]:
dirpath = os.path.join(Path.home(), "FjordsSim_data", "varna")
ds = xr.open_dataset(os.path.join(dirpath, "Varna_BRY_1lat.nc"))

In [17]:
ds = ds.drop_vars(['latitude', 'longitude'])

In [6]:
depth = np.loadtxt('Varna_topo_channels.txt')

In [24]:
# Define new dimensions
Ny_new, Nx_new = depth.shape

In [22]:
z_faces = -np.arange(0,20,1)

In [11]:
z_centers = [(z_faces[i] + z_faces[i+1]) / 2 for i in range(len(z_faces) - 1)]

In [12]:
ds = ds.interp(depth=z_centers)

In [19]:
lons = np.linspace(27.640, 27.947, depth.shape[1])
lats = np.linspace(43.177, 43.214, depth.shape[0])

In [28]:
ds

<xarray.Dataset>
Dimensions:  (time: 4019, Nx: 1, Ny: 1, Nz: 19)
Coordinates:
  * time     (time) datetime64[ns] 1989-01-01 1989-01-02 ... 2000-01-02
  * Nz       (Nz) float64 -0.5 -1.5 -2.5 -3.5 -4.5 ... -15.5 -16.5 -17.5 -18.5
Dimensions without coordinates: Nx, Ny
Data variables:
    S        (Nx, Ny, time, Nz) float64 17.98 17.98 17.98 ... 17.52 17.52 17.52
    T        (Nx, Ny, time, Nz) float64 7.084 7.089 7.094 ... 8.32 8.323 8.323
    u        (Nx, Ny, time, Nz) float64 0.0004318 0.0003695 ... 0.01169 0.01169
    v        (Nx, Ny, time, Nz) float64 0.004403 0.003689 ... 0.01439 0.01439
    zos      (Nx, Ny, time) float32 -0.1334 -0.08608 -0.0973 ... 0.2257 0.1838
Attributes:
    Conventions:               CF-1.0
    bulletin_date:             20221101
    bulletin_type:             reanalysis
    comment:                   Please check in CMEMS catalogue the INFO secti...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    field_type:                daily_mean_centered_at_time_field
    institution:               Centro Euro-Mediterraneo sui Cambiamenti Clima...
    references:                Please check in CMEMS catalogue the INFO secti...
    source:                    BS E3R1
    title:                     Salinity (3D) - Daily Mean
    copernicusmarine_version:  1.2.4

In [21]:
# Create an xarray.Dataset
ds = xr.Dataset(
    {
        "T": (["Ny", "Nx", "time", "Nz"], data),
    },
    coords={
        "time": np.arange(time),
        "Nx": np.arange(Nx),
        "Ny": np.arange(Ny),
        "depth": np.arange(depth),
    }
)

<xarray.Dataset>
Dimensions:  (time: 4019, Nx: 1, Ny: 1, depth: 19)
Coordinates:
  * time     (time) datetime64[ns] 1989-01-01 1989-01-02 ... 2000-01-02
  * depth    (depth) float64 -0.5 -1.5 -2.5 -3.5 ... -15.5 -16.5 -17.5 -18.5
Dimensions without coordinates: Nx, Ny
Data variables:
    so       (Nx, Ny, time, depth) float64 17.98 17.98 17.98 ... 17.52 17.52
    thetao   (Nx, Ny, time, depth) float64 7.084 7.089 7.094 ... 8.323 8.323
    uo       (Nx, Ny, time, depth) float64 0.0004318 0.0003695 ... 0.01169
    vo       (Nx, Ny, time, depth) float64 0.004403 0.003689 ... 0.01439 0.01439
    zos      (Nx, Ny, time) float32 -0.1334 -0.08608 -0.0973 ... 0.2257 0.1838
Attributes:
    Conventions:               CF-1.0
    bulletin_date:             20221101
    bulletin_type:             reanalysis
    comment:                   Please check in CMEMS catalogue the INFO secti...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    field_type:                daily_mean_centered_at_time_field
    institution:               Centro Euro-Mediterraneo sui Cambiamenti Clima...
    references:                Please check in CMEMS catalogue the INFO secti...
    source:                    BS E3R1
    title:                     Salinity (3D) - Daily Mean
    copernicusmarine_version:  1.2.4

In [ ]:
ds.T.isel(time=0).sel(Nz=-3).plot()

In [ ]:
ds.S.isel(time=0).sel(Nz=-3).plot()

In [ ]:
ds.S.isel(time=0).sel(Nz=-450).plot()

In [ ]:
mask = np.zeros_like(ds["T"], dtype=bool)
mask[:, :, [0, -1], :] = True
mask[:, :, :, [0, -1]] = True

In [ ]:
ds["T"] = xr.where(mask, ds["T"], np.nan)
ds["S"] = xr.where(mask, ds["S"], np.nan)
ds["u"] = xr.where(mask, ds["u"], np.nan)
ds["v"] = xr.where(mask, ds["v"], np.nan)

In [ ]:
ds.T.isel(time=0, Nz=-1).plot()

In [ ]:
ds.S.isel(time=0, Nz=-1).plot()

In [ ]:
ds = ds.fillna(-999)

In [ ]:
ds[["time", "T", "S"]].to_netcdf(os.path.join(dirpath, "Sogn_bry800.nc"))